In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data = pd.read_csv('../input/fetal-health-classification/fetal_health.csv')

### Setting option to display max columns

In [ ]:
pd.set_option('display.max_columns',None)

In [ ]:
data.head()

### Finding any null values

In [ ]:
data.isnull().sum()

### Checking datatypes of all columns

In [ ]:
data.dtypes

### Splitting data into independent and target variables

In [ ]:
X = data.iloc[:,:-1]
y = data.iloc[:,-1]

### Finding the correlation between all independent variables

In [ ]:
sns.heatmap(X.corr())

### Checking the distribution of all the continuous features

In [ ]:
for i in X.columns:
    sns.distplot(X[i],kde=False)
    plt.show()

### Performing feature scaling (Standard Scaler)

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scale = StandardScaler()
X = scale.fit_transform(X)
X = pd.DataFrame(X,columns=data.iloc[:,:-1].columns)

### Checking if the dataset is balanced or not

In [ ]:
data.fetal_health.value_counts()

### Using oversampling technique for imbalanced dataset

In [ ]:
from imblearn.over_sampling import RandomOverSampler as ROS

In [ ]:
ros = ROS(random_state=42)

In [ ]:
X_res, y_res = ros.fit_resample(X,y)

In [ ]:
from collections import Counter
print('Resampled dataset shape %s' % Counter(y_res))

### Performing train test split

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_res,y_res,test_size=0.3)

### SVM with GridSearchCV

In [ ]:
from sklearn.svm import SVC

In [ ]:
model = SVC(decision_function_shape='ovo')

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = [{'C': [1, 10, 100, 1000], 'kernel': ['linear']},
              {'C': [1, 10, 100, 1000], 'kernel': ['rbf'], 'gamma': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]}]
grid_search = GridSearchCV(estimator = model,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10,
                           n_jobs = -1)
grid_search = grid_search.fit(X_train, y_train)

In [ ]:
accuracy = grid_search.best_score_
accuracy

In [ ]:
grid_search.best_params_

In [ ]:
model = SVC(C=1000,gamma=0.9,kernel='rbf',decision_function_shape='ovo',probability=True)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
pred = model.predict(X_test)

### Finding accuracy score, auc score, f1 score for kernal SVM

In [ ]:
from sklearn.metrics import confusion_matrix as cm

In [ ]:
cm(y_test,pred)

In [ ]:
from sklearn.metrics import accuracy_score as acc

In [ ]:
acc(y_test,pred)

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve, f1_score, auc

In [ ]:
f1_score(y_test,pred,average='micro')

In [ ]:
probs = model.predict_proba(X_test)

In [ ]:
roc_auc_score(y_test,probs,multi_class='ovo')

### XGBoost with GridSearchCV

In [ ]:
from xgboost import XGBClassifier

In [ ]:
model = XGBClassifier()

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = [{'max_depth':[3,6,9,12], 'learning_rate':[0.1],'n_estimators':[50,100,200]},
              {'max_depth':[3,6,9,12], 'learning_rate':[0.01],'n_estimators':[50,100,200]}]
grid_search = GridSearchCV(estimator = model,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10,
                           n_jobs = -1)
grid_search = grid_search.fit(X_train, y_train)

In [ ]:
grid_search.best_params_

In [ ]:
accuracy = grid_search.best_score_
accuracy

In [ ]:
model = XGBClassifier(max_depth=12,n_estimators=200)
model.fit(X_train,y_train)
pred = model.predict(X_test)

### Calculating accuracy, f1-score and auc score for xgboost ensamble technique

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve, f1_score, auc, confusion_matrix as cm, accuracy_score as acc
print('confusion\n',cm(y_test,pred))
print('\n')
print('accuracy',acc(y_test,pred))
print('\n')
print('f1',f1_score(y_test,pred,average='micro'))
print('\n')
probs = model.predict_proba(X_test)
print('auc',roc_auc_score(y_test,probs,multi_class='ovo'))
print('\n')